In [1]:
import os
import pandas as pd

from pathlib import Path
from glob import glob
from datetime import datetime

!pip install xlrd

Pour harmoniser les importations en chemin relatifs, dans des fichiers cousins :

In [5]:
current_path = Path(os.getcwd())
root = current_path.parent.absolute()
data_folder = str(root) + '\\treated_data\\'

d:\Github\VIX\VIX_predictions-1\treated_data\


In [8]:
types = ["xlsx", "csv", "xls"]

os.chdir(data_folder)

sheets_dict = {}

print("-- XLSX FILES --")
for file_path in glob("*.xlsx"):
    file_name = file_path.split(".")[0]


    print(file_name + " sheets:")
    
    file = pd.ExcelFile(file_path)
    for sheet in file.sheet_names:
        print("-->"+sheet)
        sheets_dict[sheet] = file.parse(sheet)
    print()
    
    
print("\n-- XLS FILES --") 
for file_path in glob("*.xls"):
    file_name = file_path.split(".")[0]

    print(file_name + " sheets:")
    
    file = pd.ExcelFile(file_path)
    for sheet in file.sheet_names:
        print("-->"+sheet)
        sheets_dict[sheet] = file.parse(sheet)
    print()
    
print("\n-- csv FILES --")
for file_path in glob("*.csv"):
    file_name = file_path.split(".")[0]
    print(file_name)
    file_df = pd.read_csv(file_path, sep=';')
    sheets_dict[file_name] = file_df


-- XLSX FILES --
Betting Against Beta Equity Factors Daily sheets:
-->BAB Factors

bloom_data_2 sheets:
-->CL1
-->CL2
-->CL3
-->Euribor3
-->Bund10
-->FUTURES MATURITIES

Quality Minus Junk Factors Daily sheets:
-->QMJ Factors
-->HML Devil


In [7]:

# Intersect avec dates VIX
# Donnés monthly à intersect
# supprimer value and momentum

keys = list(sheets_dict.keys())
for key in keys:
    if key !='FUTURES MATURITIES':
        if type(sheets_dict[key].index)!=pd.DatetimeIndex:
            date_col = [col for col in ['Date','DATE','Unnamed: 0','Reported data', 'time'] if col in sheets_dict[key].columns]
            assert len(date_col)==1, "Date column is not unique"+ str( key)
            if key in ['F-F_ST_Reversal_Factor_daily', 'F-F_Research_Data_5_Factors_2x3_daily', 'F-F_Momentum_Factor_daily']:
                sheets_dict[key][date_col]= sheets_dict[key][date_col].applymap(lambda x: datetime.strptime(x,'%d/%m/%Y'))
            sheets_dict[key].set_index(date_col, inplace=True)
            if key == 'VME Factors':
                # Resampling monthly -> daily
                #The value for month n is the one released on the last day of the month n-1, as it is one agents use during month n.
                start_date = sheets_dict["VME Factors"].index.min() - pd.DateOffset(day=1)
                end_date = sheets_dict["VME Factors"].index.max() + pd.DateOffset(day=31)
                dates = pd.date_range(start_date, end_date, freq='D')
                sheets_dict["VME Factors"] = sheets_dict["VME Factors"].reindex(dates, method='ffill')
        sheets_dict[key].index.names = ["Date"]
        #Adding a suffix
        sheets_dict[key].rename(columns = lambda x: x+'_'+key,inplace=True)


AssertionError: Date column is not uniquevolatility

In [6]:
keys = sheets_dict.keys()
print(keys)
# final_base = pd.merge_asof(sheets_dict[keys[0]], sheets_dict[keys[1]], on="Date")

#     big_base = pd.merge_asof(big_base, index_n_vol['VSTOXX'], on="Date")

dict_keys(['BAB Factors', 'CL1', 'CL2', 'CL3', 'Euribor3', 'Bund10', 'FUTURES MATURITIES', 'QMJ Factors', 'HML Devil', 'Sheet1', 'SENTIMENT', 'F-F_Momentum_Factor_daily', 'F-F_Research_Data_5_Factors_2x3_daily', 'F-F_ST_Reversal_Factor_daily'])


In [9]:
os.chdir(root)
big_base.to_excel(str(root) + 'database.xlsx', na_rep='#N/A N/A')


NameError: name 'big_base' is not defined